In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.cluster import KMeans,AgglomerativeClustering
from sklearn.metrics import silhouette_score

In [ ]:
data=pd.read_csv("/kaggle/input/unsupervised-learning-on-country-data/Country-data.csv")
data.head()

# Data Cleaning

In [ ]:
data.shape

In [ ]:
data.isna().sum()

In [ ]:
data.drop_duplicates(inplace=True)

# EDA
### Univariate Analysis

In [ ]:
fig,axs=plt.subplots(3,3,figsize=(20,13))
col=['child_mort', 'exports', 'health', 'imports', 'income','inflation', 'life_expec', 'total_fer', 'gdpp']
ax=axs.flatten()
for i,j in enumerate(col):
    sns.boxplot(x=data[j],ax=ax[i],color='red')

### Bivariate Analysis

In [ ]:
sns.pairplot(data,corner=True)

### Correlation

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(data.corr(),annot=True,square=True)

# Data Preparation for modelling

In [ ]:
data1=data.drop('country',axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
s=StandardScaler()
data1=s.fit_transform(data1);

# Data Modelling

### K-Means

In [ ]:
#Elbow plot
from sklearn.cluster import KMeans
inertia=[]
for i in range(1,10):
    k=KMeans(i)
    k.fit(data1)
    inertia.append(k.inertia_)
plt.plot(range(1,10),inertia,marker='o')

In [ ]:
#From elbow plot, k=3,4 are good because of large difference errors...to decide one in these two, we go for silhouette_score
score=[]
plt.figure(figsize=(10,10))
for i in range(2,10):
    k=KMeans(i)
    k.fit(data1)
    score.append(np.round(silhouette_score(data1,k.labels_),2))
plt.plot(range(2,10),score)

In [ ]:
#k=4 is giving high silhouette score than k=3...even though k=5 is giving high silhouette_score, the no. of clusters should not be high, so we go for k=4.
k=KMeans(4,random_state=42)
k.fit(data1)
data['k_labels']=k.labels_

### Hierarchial Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
score=[]
for i in range(2,10):
    a=AgglomerativeClustering(i)
    a.fit(data1)
    score.append(np.round(silhouette_score(data1,a.labels_),2))
plt.plot(range(2,10),score)

In [ ]:
#k=2 gives good silhouette score
a=AgglomerativeClustering(2)
a.fit(data1)
data['hier_labels']=a.labels_

In [ ]:
data.head()

### Replacing labels with 'Help needed' or 'not'

In [ ]:
data.drop('hier_labels',axis=1).groupby(['k_labels','country']).mean()

In [ ]:
def func(x):
    if x==0:
        return 'Help needed priority-1' 
    elif x==1:
        return 'Help needed priority-2'
    elif x==2:
        return 'Help needed priority-3'
    else:
        return 'No Help needed'
data['k_labels']=data['k_labels'].map(lambda x: func(x))

In [ ]:
data.drop('k_labels',axis=1).groupby(['hier_labels','country']).mean()

In [ ]:
def func(x):
    if x==0:
        return 'Help needed'
    else:
        return 'No Help needed'
data['hier_labels']=data['hier_labels'].map(lambda x: func(x))

In [ ]:
data.head(20)

## Output

In [ ]:
print('Based on K-Means clustering, the countries which need Help the most(on priority-1) are:')
data.loc[data['k_labels']=='Help needed priority-1']['country'].to_list()

In [ ]:
print('Based on Hierarchial clustering, the countries which need Help are:')
data.loc[data['hier_labels']=='Help needed']['country'].to_list()